In [99]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from folktables import ACSDataSource, ACSEmployment, generate_categories, ACSIncome, ACSHealthInsurance
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics.regressionplots import abline_plot
from statsmodels.formula.api import ols
from scipy.stats import linregress
import math
import sklearn
from sklearn.metrics import mean_squared_error
import statistics
import numpy as np

In [75]:
from folk_data_preProcessor import preProcessor
from folktables import ACSDataSource, ACSEmployment

# import the data
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)
features, label, _ = ACSEmployment.df_to_pandas(acs_data)

# obtain the data sets
data = preProcessor(acs_data, features, "WAGP")

In [21]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)
features, label, group = ACSEmployment.df_to_numpy(acs_data)
nat_group = acs_data["NATIVITY"]
sex_group = acs_data["SEX"]
cont_labels = acs_data["WAGP"]
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features, label, group, test_size=0.2, random_state=np.random.seed())

#X_train, X_test, y_train, y_test, group_train, group_test, sex_train, sex_test = train_test_split(features, label, group, sex_group, test_size=0.2, random_state=np.random.seed())

#### Linear Regression

In [112]:
#linear
import scipy
linreg=LinearRegression()
linreg.fit(X_train,y_train)
y_predn=linreg.predict(X_test)
from sklearn import metrics
print("RMSE: %f" % (metrics.mean_squared_error(y_test,y_predn, squared=False)))

RMSE: 0.402316


#### Using XGBoost

In [8]:
import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
print(xgboost.__version__)


1.6.2


References:
1. https://machinelearningmastery.com/xgboost-for-regression/
2. https://www.youtube.com/watch?v=sxfpzUF6FLA


In [9]:
#non-linear
xg_reg = xgboost.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, learning_rate = 0.6,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.347192


### Note: currently the part from here to AIF360 is not in use

In [ ]:
#not currently in use
features, label, group = ACSEmployment.df_to_numpy(ca_features)
#nat_group = ca_features["NATIVITY"]

dff = ca_data[ca_data['WAGP'].notna()]
t = pd.DataFrame(np.hstack((features, np.array(cont_labels).reshape(-1, 1)))).dropna()
cont_features, cont_labels, cont_group = t.iloc[:, 0:16], t.iloc[:,16], t[15]
#caa_data=ca_data.drop(['WAGP'],axis=1)


In [ ]:
ACSEmployment.df_to_numpy(ca_data)

In [ ]:
ca_use=pd.DataFrame(ca_data)

ca_use=ca_use.drop(['RT', 'SERIALNO', 'NAICSP', 'SOCP','WAGP'], axis=1)
ca_use.info()

In [ ]:
ca_use=pd.DataFrame(dff)

ca_use=ca_use.drop(['RT', 'SERIALNO', 'NAICSP', 'SOCP','WAGP','CITWP','COW'], axis=1)
ca_use.head()

In [ ]:
X,y=cont_features,cont_labels
X

In [ ]:
data_dmatrix = xgboost.DMatrix(data=X,label=y)
data_dmatrix

# AIF360

Issue 1: requires to convert the dataframe to a BinaryLabelDataSet

In [12]:
un=acs_data.dropna(how='any',axis=1)
un.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378817 entries, 0 to 378816
Columns: 199 entries, RT to PWGTP80
dtypes: int64(197), object(2)
memory usage: 575.1+ MB


In [113]:
target_sex=un['SEX'].value_counts()
target_sex

2    192140
1    186677
Name: SEX, dtype: int64

In [ ]:
target_race=un['RAC1P'].value_counts()
target_race

In [ ]:
target_status=un['MAR'].value_counts()
target_status

Dropping some of those columns from the dataset doesn't change anything, it just helps process 3 times faster. I will standardize the PWGTP ones .

In [ ]:
un=un.drop(['RT','SERIALNO'])

In [28]:
un=un.drop(['PWGTP50','PWGTP51','PWGTP52','PWGTP53','PWGTP54','PWGTP55','PWGTP56','PWGTP57','PWGTP58','PWGTP59','PWGTP60','PWGTP61','PWGTP62','PWGTP63','PWGTP64','PWGTP65','PWGTP66','PWGTP67','PWGTP68','PWGTP69','PWGTP70','PWGTP71','PWGTP72','PWGTP73','PWGTP74','PWGTP75','PWGTP76','PWGTP77','PWGTP78','PWGTP79','PWGTP80' ], axis=1)

In [30]:
un=un.drop(['PWGTP20','PWGTP21','PWGTP22','PWGTP23','PWGTP24','PWGTP25','PWGTP26','PWGTP27','PWGTP28','PWGTP29','PWGTP30','PWGTP31','PWGTP32','PWGTP33','PWGTP34','PWGTP35','PWGTP36','PWGTP37','PWGTP38','PWGTP39','PWGTP40','PWGTP41','PWGTP42','PWGTP43','PWGTP44','PWGTP45','PWGTP46','PWGTP47','PWGTP48','PWGTP49'], axis=1)

In [32]:
un=un.drop(['PWGTP10','PWGTP11','PWGTP12','PWGTP13','PWGTP14','PWGTP15','PWGTP16','PWGTP17','PWGTP18','PWGTP19'], axis=1)

In [35]:
un=un.drop(['PWGTP1','PWGTP2','PWGTP3','PWGTP4','PWGTP5','PWGTP6','PWGTP7','PWGTP8','PWGTP9'], axis=1)

In [47]:
un=un.drop(['DIVISION', 'REGION','ST','ADJINC'],axis=1)

In [66]:
un['FRACP'].value_counts()

0    367906
1     10911
Name: FRACP, dtype: int64

In [ ]:
# this is using the dataset from the import file (the one Oswaldo and I were gonna use to make sure we have the same data)
"""binaryLabelDataset_processed = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=2,
    df=data,
    label_names=['WAGP'],
    protected_attribute_names=['SEX'])"""

In [67]:
import aif360
from aif360.algorithms.preprocessing import DisparateImpactRemover
# This is what would have been used if we were evaluating for two outcomes - either success or failure. I ended up figuring out the other one and used it successfully.
binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=un,
    label_names=['FWAGP'],
    protected_attribute_names=['SEX'])

In [102]:
sd = aif360.datasets.StructuredDataset(df=un, label_names=['FWAGP'], protected_attribute_names=['SEX'])

In [103]:
sd

               instance weights features                                      \
                                                                               
                                 SPORDER    PUMA  PWGTP  AGEP  CIT DEAR DEYE   
instance names                                                                 
0                           1.0      1.0  3701.0   32.0  30.0  1.0  2.0  2.0   
1                           1.0      1.0  7306.0   45.0  18.0  1.0  2.0  2.0   
2                           1.0      1.0  3755.0  109.0  69.0  1.0  2.0  2.0   
3                           1.0      1.0  7319.0   34.0  25.0  1.0  1.0  2.0   
4                           1.0      1.0  6511.0   46.0  31.0  1.0  2.0  2.0   
...                         ...      ...     ...    ...   ...  ...  ...  ...   
378812                      1.0      2.0  1308.0   94.0  61.0  4.0  2.0  2.0   
378813                      1.0      1.0  7108.0  172.0  40.0  4.0  2.0  2.0   
378814                      1.0      2.0

In [70]:
un.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378817 entries, 0 to 378816
Columns: 113 entries, SPORDER to FYOEP
dtypes: int64(113)
memory usage: 326.6 MB


In [72]:
di = DisparateImpactRemover(repair_level = 1.0)
dataset_transf_train = di.fit_transform(binaryLabelDataset)
transformed = dataset_transf_train.convert_to_dataframe()[0]

In [104]:
#for Structured Dataset
di1 = DisparateImpactRemover(repair_level = 1.0)
dataset_transf_train1 = di.fit_transform(sd)
transformed = dataset_transf_train1.convert_to_dataframe()[0]

In [105]:
x_trans = transformed.drop(['FWAGP'], axis = 1)
y = transformed['FWAGP']
scaler = StandardScaler()
data_std = scaler.fit_transform(x_trans)
x_trans_train,x_trans_test,y_trans_train,y_trans_test = train_test_split(x_trans, y, test_size=0.2, random_state = 0)

In [106]:
y_trans_pred=model.fit(x_trans_train,y_trans_train)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [108]:
y_transpred=y_trans_pred.predict(x_trans_test)

In [110]:
print("RMSE: %f" % (metrics.mean_squared_error(y_trans_test,y_transpred, squared=True)))

RMSE: 0.082612


In [114]:
from sklearn import metrics 
print("Accuracy:", metrics.accuracy_score(y_test,y_transpred))
print("Precision:",metrics.precision_score(y_test,y_transpred))
print("Recall:", metrics.recall_score(y_test,y_transpred))

Accuracy: 0.538065572039491
Precision: 0.4598093612178119
Recall: 0.09386344495106438
